# Import

In [31]:
import pandas as pd

In [32]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [33]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")  
submission = pd.read_csv("./data/sample_submission.csv")  

영어 번역 내용 제거

In [34]:
import os
import pandas as pd
from datetime import datetime
import re

# '키워드' 열에서 '영문', '번역', '오류', '전제' 네 단어를 모두 포함하는 행을 찾는 조건
condition_train_1 = train_df['키워드'].apply(lambda x: all(word in x for word in ['영문', '번역', '오류', '전제']))
condition_test_1 = test_df['키워드'].apply(lambda x: all(word in x for word in ['영문', '번역', '오류', '전제']))

# '키워드' 열에서 '구글', '번역기', '번역', '영문기사' 네 단어를 모두 포함하는 행을 찾는 조건
condition_train_2 = train_df['키워드'].apply(lambda x: all(word in x for word in ['구글', '번역기', '번역', '영문기사']))
condition_test_2 = test_df['키워드'].apply(lambda x: all(word in x for word in ['구글', '번역기', '번역', '영문기사']))

# '키워드' 열에서 '기사'와 '구글' 두 단어를 모두 포함하는 행을 찾는 조건
condition_train_3 = train_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))
condition_test_3 = test_df['키워드'].apply(lambda x: all(word in x for word in ['기사', '구글']))

# '키워드' 열에서 'Break' 단어를 포함하는 행을 찾는 조건
condition_train_4 = train_df['키워드'].str.contains(r'\bBreak\b', na=False)
condition_test_4 = test_df['키워드'].str.contains(r'\bBreak\b', na=False)

# 조건을 만족하는 행들로 새로운 데이터프레임 생성
filtered_train_df_1 = train_df[condition_train_1].copy()
filtered_test_df_1 = test_df[condition_test_1].copy()
filtered_train_df_2 = train_df[condition_train_2].copy()
filtered_test_df_2 = test_df[condition_test_2].copy()
filtered_train_df_3 = train_df[condition_train_3].copy()
filtered_test_df_3 = test_df[condition_test_3].copy()
filtered_train_df_4 = train_df[condition_train_4].copy()
filtered_test_df_4 = test_df[condition_test_4].copy()

# '영문, 번역, 오류, 전제' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords_1(text):
    keywords = ['영문', '번역', '오류', '전제']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('영문')].strip()

# '구글, 번역기, 번역, 영문기사' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords_2(text):
    keywords = ['구글', '번역기', '번역', '영문기사']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('구글')].strip()

# '기사, 구글' 순서로 단어가 나오는 부분을 찾아 삭제하는 함수 정의
def remove_keywords_3(text):
    keywords = ['기사', '구글']
    start_index = 0
    for keyword in keywords:
        start_index = text.find(keyword, start_index)
        if start_index == -1:
            return text
        start_index += len(keyword)
    return text[:text.find('기사')].strip()

# 한글 단어가 나오다가 영어 단어가 5번 이상 나오면 영어 단어가 나오기 시작한 단어부터 끝까지 삭제하는 함수 정의
def remove_english_after_five(text):
    # 영어 단어를 찾는 정규 표현식
    english_words = re.findall(r'[a-zA-Z]+', text)
    if len(english_words) >= 5:
        # 5번째 영어 단어의 시작 위치 찾기
        fifth_english_word = english_words[0]
        start_index = text.find(fifth_english_word)
        return text[:start_index].strip()
    return text

# 각 행의 '키워드' 값에서 해당 단어부터 마지막 단어까지 삭제
filtered_train_df_1['키워드'] = filtered_train_df_1['키워드'].apply(remove_keywords_1)
filtered_test_df_1['키워드'] = filtered_test_df_1['키워드'].apply(remove_keywords_1)
filtered_train_df_2['키워드'] = filtered_train_df_2['키워드'].apply(remove_keywords_2)
filtered_test_df_2['키워드'] = filtered_test_df_2['키워드'].apply(remove_keywords_2)
filtered_train_df_3['키워드'] = filtered_train_df_3['키워드'].apply(remove_keywords_3)
filtered_test_df_3['키워드'] = filtered_test_df_3['키워드'].apply(remove_keywords_3)
filtered_train_df_4['키워드'] = filtered_train_df_4['키워드'].apply(remove_english_after_five)
filtered_test_df_4['키워드'] = filtered_test_df_4['키워드'].apply(remove_english_after_five)

# 수정된 '키워드' 값을 원래 데이터프레임에 반영
train_df.loc[condition_train_1, '키워드'] = filtered_train_df_1['키워드']
test_df.loc[condition_test_1, '키워드'] = filtered_test_df_1['키워드']
train_df.loc[condition_train_2, '키워드'] = filtered_train_df_2['키워드']
test_df.loc[condition_test_2, '키워드'] = filtered_test_df_2['키워드']
train_df.loc[condition_train_3, '키워드'] = filtered_train_df_3['키워드']
test_df.loc[condition_test_3, '키워드'] = filtered_test_df_3['키워드']
train_df.loc[condition_train_4, '키워드'] = filtered_train_df_4['키워드']
test_df.loc[condition_test_4, '키워드'] = filtered_test_df_4['키워드']

In [35]:
# 수정된 데이터프레임을 CSV 파일로 저장
train_df.to_csv('./data/train_df_translate_del.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('./data/test_df_translate_del.csv', index=False, encoding='utf-8-sig')

.